<div style="border:solid green 2px; padding: 20px"> <h1 style="color:green; margin-bottom:20px">Комментарий наставника</h1>

Привет! Поздравляю тебя с первым твоим проектом и спасибо, что сдала задание:) Ты проделала большую работу. Далее в файле мои комментарии ты сможешь найти в ячейках, аналогичных данфной ( если рамки комментария зелёные - всё сделано правильно; жёлтые - есть замечания, но не критично; красные - нужно переделать). Не удаляй эти комментарии и постарайся учесть их в ходе выполнения проекта. 

# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

Изучим общую информацию с данными:

In [1]:
import pandas as pd
data = pd.read_csv('/datasets/data.csv')
print(data.head(10))
#посмотрим первые 10 строчек таблицы

   children  days_employed  dob_years education  education_id  \
0         1   -8437.673028         42    высшее             0   
1         1   -4024.803754         36   среднее             1   
2         0   -5623.422610         33   Среднее             1   
3         3   -4124.747207         32   среднее             1   
4         0  340266.072047         53   среднее             1   
5         0    -926.185831         27    высшее             0   
6         0   -2879.202052         43    высшее             0   
7         0    -152.779569         50   СРЕДНЕЕ             1   
8         2   -6929.865299         35    ВЫСШЕЕ             0   
9         0   -2188.756445         41   среднее             1   

      family_status  family_status_id gender income_type  debt   total_income  \
0   женат / замужем                 0      F   сотрудник     0  253875.639453   
1   женат / замужем                 0      F   сотрудник     0  112080.014102   
2   женат / замужем                 0    

In [2]:
print(data.tail(10))
# посмотрим последние 10 строчек таблицы

       children  days_employed  dob_years       education  education_id  \
21515         1    -467.685130         28         среднее             1   
21516         0    -914.391429         42          высшее             0   
21517         0    -404.679034         42          высшее             0   
21518         0  373995.710838         59         СРЕДНЕЕ             1   
21519         1   -2351.431934         37  ученая степень             4   
21520         1   -4529.316663         43         среднее             1   
21521         0  343937.404131         67         среднее             1   
21522         1   -2113.346888         38         среднее             1   
21523         3   -3112.481705         38         среднее             1   
21524         2   -1984.507589         40         среднее             1   

          family_status  family_status_id gender income_type  debt  \
21515   женат / замужем                 0      F   сотрудник     1   
21516   женат / замужем           

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Выводы:
Наблюдаем следующие возможные сложности в дальнейшем анализе: 

- отрицательные значения в столбцах children и days_employed
- наличие NAN в столбцах days_employed и total_income
- прыгающий регистр букв в столбце education
- есть дублирующие признаки
- непонятна валюта месячного дохода

<div style="border:solid green 2px; padding: 20px"> <h1 style="color:green; margin-bottom:20px">Комментарий наставника</h1>

Хорошее начало, так же, для большего понимания данных можно было бы использовать метод `describe`


### Шаг 2. Предобработка данных

### Обработка пропусков

In [4]:
#посмотрим на суммарное количество пропусков
data.isnull().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [5]:
#заменим пропуски в столбце total_income, 
#так как в дальнейшем он понадобится для анализа данных 
#по запрошенным задачам
data['total_income'] = data['total_income'].fillna(0)

In [6]:
data['days_employed'] = data['days_employed'].fillna(0)

In [7]:
#проверяем, остались ли пропуска после замены
data.isnull().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

# Вывод
- какие пропущенные значения обнаружили:
Пропущенные значения обнаружены в двух столбцах: days_employed и total_income (в 2174 строках из 21525 строках пропущены значения).

- возможные причины появления пропусков в данных: 
1. люди намерено пропустили эти пункты, потому что работают не официально;
2. еще просто не работали
3. баг, данные некоректно введены

- объясните, по какому принципу заполнены пропуски:
Был применен метод, который подразумевает заполнение NaN на 0, применять в данном случае метод mean() или mead() было бы некорректным, это могло бы исказить данные, также как и удаление пропусков. 

Далее будем рассматривать в анализе столбец total_income, так как именно он понадобится для решение поставленных клиентом задач. 

<div style="border:solid orange 2px; padding: 20px"> <h1 style="color:orange; margin-bottom:20px">Комментарий наставника</h1>

Тут лучше не заполнять пропуски просто медианой, средним или нулём, требуется более обоснованное заполнение пропусков исходя из имеющихся данных

### Замена типа данных

В запросе заказчика есть следующие вопросы: 
1. Есть ли зависимость между наличием детей и возвратом кредита в срок? 
2. Есть ли зависимость между семейным положением и возвратом кредита в срок? 
3. Есть ли зависимость между уровнем дохода и возвратом кредита в срок? 
4. Как разные цели кредита влияют на его возврат в срок?

В связи с этим, далее будем рассматривать и перерабатывать данные только в необходимых для анализа столбцах. Если потребуется дополнительная информация для анализа из оставшихся столбцах, тогда заменим типы данных и там =)

In [8]:
#посмотрим столбец "children", так как есть запрос: 
#влияет ли наличие детей не возврат кредита в срок.
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

In [9]:
data[data['children'] == -1]['children'].count()

47

In [10]:
#-1 можно попробовать заменить на среднее количество детей,
#так как не факт, что имелось ввиду количество одного ребенка. 
mean_child = data['children'].mean()
print(mean_child)

0.5389082462253194


In [11]:
#переведем в целое число для удобства
mean_child1 = int(mean_child*1000000)
print(type(mean_child1))

<class 'int'>


In [12]:
#Заменим -1 на значение mean_child1
data['children'] = data['children'].replace(-1, mean_child1)

In [13]:
data[data['children'] == -1]['children'].count()

0

In [42]:
#Посмотрим сколько строк, где количество детей = 20
data[data['children'] == 20]['children'].count()

76

In [43]:
data['children'].astype('int')

0        1
1        1
2        0
3        3
4        0
        ..
21466    1
21467    0
21468    1
21469    3
21470    2
Name: children, Length: 21471, dtype: int64

В постановке задачи не стоит вопрос о количестве детей, стоит вопрос влияет ли наличие детей на возврат кредита в срок? Соотвтетсвенно, нас интересует ответ на вопрос есть дети у клиента или нет. Далее в проекте распределим эти данные на категории. 

In [16]:
#теперь посмотрим столбец "family_status" для дальнейшего анализа 
data['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

In [17]:
#приведем все значения к нижнему регистру
data['family_status'] = data['family_status'].str.lower()
data['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'не женат / не замужем'], dtype=object)

In [18]:
#рассмотрим уровень дохода
print(data['total_income'])

0        253875.639453
1        112080.014102
2        145885.952297
3        267628.550329
4        158616.077870
             ...      
21520    224791.862382
21521    155999.806512
21522     89672.561153
21523    244093.050500
21524     82047.418899
Name: total_income, Length: 21525, dtype: float64


In [19]:
#приведем данные в более удобный для анализа вид
data['total_income'] = data['total_income'].astype(int)
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи


In [20]:
#посмотрим на следующий стобец для дальнейшего анализа "purpose"
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

In [21]:
#dept
data['debt'].value_counts()

0    19784
1     1741
Name: debt, dtype: int64

В блоке "Замена типов данных" мы рассмотрели все необходимые для дальнейшего анализа поставленных вопросов заказчиком столбцы. 
Необходимо доработать стобец - "purpose" (далее будет к нему применен метод лемматизации), 

<div style="border:solid green 2px; padding: 20px"> <h1 style="color:green; margin-bottom:20px">Комментарий наставника</h1>

Замена типа данных выполнена верно

### Обработка дубликатов

In [22]:
#проверим наличие дубликатов
data.duplicated().sum()

54

Возможо дубли появились при заполнении данных, удалим и проверим заново дубли.

In [23]:
data = data.drop_duplicates().reset_index(drop=True)

In [24]:
data.duplicated().sum()

0

In [25]:
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи


### Вывод
Проверили на наличие дубликатов, которые, возможно, возникли из-за неправильного заполнения данных. 
Дубликаты удалили и заново проверили  их на наличие. Были применены методы duplicated() и drop_duplicates().

<div style="border:solid green 2px; padding: 20px"> <h1 style="color:green; margin-bottom:20px">Комментарий наставника</h1>

Дубликаты найдены и обработаны верно,отлично


### Лемматизация

In [26]:
#Для начала выведем еще раз список значений в столбце "purpose"
data['purpose'].unique().tolist()

['покупка жилья',
 'приобретение автомобиля',
 'дополнительное образование',
 'сыграть свадьбу',
 'операции с жильем',
 'образование',
 'на проведение свадьбы',
 'покупка жилья для семьи',
 'покупка недвижимости',
 'покупка коммерческой недвижимости',
 'покупка жилой недвижимости',
 'строительство собственной недвижимости',
 'недвижимость',
 'строительство недвижимости',
 'на покупку подержанного автомобиля',
 'на покупку своего автомобиля',
 'операции с коммерческой недвижимостью',
 'строительство жилой недвижимости',
 'жилье',
 'операции со своей недвижимостью',
 'автомобили',
 'заняться образованием',
 'сделка с подержанным автомобилем',
 'получение образования',
 'автомобиль',
 'свадьба',
 'получение дополнительного образования',
 'покупка своего жилья',
 'операции с недвижимостью',
 'получение высшего образования',
 'свой автомобиль',
 'сделка с автомобилем',
 'профильное образование',
 'высшее образование',
 'покупка жилья для сдачи',
 'на покупку автомобиля',
 'ремонт жилью',
 '

In [27]:
#Вручную выделим основные цели взятия кредита:
   # - покупка или ремонт автомобиля
   # - жилье или недвижимость (как коммерческая так и для собсвенного пользования)
   # - получение дополнительного образования
   # - свадьба
#Получается в данной таблице представлено 4 цели на взятие кредита: автомобиль, жилье/недвижимость, образование и свадьба. 

In [28]:
from pymystem3 import Mystem
from collections import Counter
m = Mystem()

def lemmatize_it(row):
        lemmas = m.lemmatize(row)
        if 'автомобиль' in lemmas:
            return 'автокредит'
        if 'свадьба' in lemmas:
            return 'кредит на свадьбу'
        if 'жилье' in lemmas or 'недвижимость' in lemmas:
            return 'ипотека'
        if 'образование' in lemmas:
            return 'кредит на образование'
        
data['purpose_group'] = data['purpose'].apply(lemmatize_it)

In [29]:
#почему то RUN не срабатывает для принта: print(data['purpose_group']), 
#поэтому пришлось выводить таким образом =)
print(data['purpose'].apply(lemmatize_it))

0                      ипотека
1                   автокредит
2                      ипотека
3        кредит на образование
4            кредит на свадьбу
                 ...          
21466                  ипотека
21467               автокредит
21468                  ипотека
21469               автокредит
21470               автокредит
Name: purpose, Length: 21471, dtype: object


In [30]:
print(data['purpose_group'])

0                      ипотека
1                   автокредит
2                      ипотека
3        кредит на образование
4            кредит на свадьбу
                 ...          
21466                  ипотека
21467               автокредит
21468                  ипотека
21469               автокредит
21470               автокредит
Name: purpose_group, Length: 21471, dtype: object


In [31]:
purpose_data = data[['purpose', 'purpose_group']]
print(purpose_data.head(10))

                      purpose          purpose_group
0               покупка жилья                ипотека
1     приобретение автомобиля             автокредит
2               покупка жилья                ипотека
3  дополнительное образование  кредит на образование
4             сыграть свадьбу      кредит на свадьбу
5               покупка жилья                ипотека
6           операции с жильем                ипотека
7                 образование  кредит на образование
8       на проведение свадьбы      кредит на свадьбу
9     покупка жилья для семьи                ипотека


### Вывод

Выделили основные леммы, а именно цели для преоретение кредита: 
- ипотека
- кредит на образование
- кредит на свадьбу
- автокредит

<div style="border:solid green 2px; padding: 20px"> <h1 style="color:green; margin-bottom:20px">Комментарий наставника</h1>


Лемматизация проведена верно

### Категоризация данных

Из необходимых для дальнейшего анализа столбцов, для категоризации данных можно рассмотреть два: children и total_income. 


In [32]:
#Категоризуем столбец "total_income", 
#для начала посмотрим на данные этого столбца
data['total_income'].value_counts()

0         2120
204827       3
163060       3
144533       3
121931       3
          ... 
109583       1
101387       1
138249       1
280240       1
229304       1
Name: total_income, Length: 18607, dtype: int64

In [33]:
#Из этого списка сложно сделать анализ, 
#поэтому необходимо разбить доход на категории по статусу:
def income_category(total_income):
    if total_income <= 45000.0:
        return 'низкий уровень дохода'
    if total_income <= 90000.0:
        return 'средний уровень дохода'
    if total_income <= 180000.0: 
        return 'высокий уровень дохода'
    return 'очень высокий уровень дохода'

data['income_category'] = data['total_income'].apply(income_category)

In [34]:
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_group,income_category
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,ипотека,очень высокий уровень дохода
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автокредит,высокий уровень дохода
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,ипотека,высокий уровень дохода
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,кредит на образование,очень высокий уровень дохода
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,кредит на свадьбу,высокий уровень дохода
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,ипотека,очень высокий уровень дохода
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,ипотека,очень высокий уровень дохода
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823,образование,кредит на образование,высокий уровень дохода
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,кредит на свадьбу,высокий уровень дохода
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,ипотека,высокий уровень дохода


In [35]:
#теперь можно посмотреть сколько людей в каждой категории: 21471 из 21525
data['income_category'].value_counts()

высокий уровень дохода          9581
очень высокий уровень дохода    6422
средний уровень дохода          3131
низкий уровень дохода           2337
Name: income_category, dtype: int64

In [36]:
#Теперь посмотрим стобец "children"
def children_category(children):
    if children <= 0:
        return 'нет детей'
    if children >= 0:
        return 'есть дети'

data['children_category'] = data['children'].apply(children_category)
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_group,income_category,children_category
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,ипотека,очень высокий уровень дохода,есть дети
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автокредит,высокий уровень дохода,есть дети
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,ипотека,высокий уровень дохода,нет детей
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,кредит на образование,очень высокий уровень дохода,есть дети
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,кредит на свадьбу,высокий уровень дохода,нет детей
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,ипотека,очень высокий уровень дохода,нет детей
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,ипотека,очень высокий уровень дохода,нет детей
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823,образование,кредит на образование,высокий уровень дохода,нет детей
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,кредит на свадьбу,высокий уровень дохода,есть дети
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,ипотека,высокий уровень дохода,нет детей


In [37]:
#теперь можно посмотреть сколько людей в каждой категории: 21471 из 21525
data['children_category'].value_counts()

нет детей    14107
есть дети     7364
Name: children_category, dtype: int64

### Вывод

Разбили на категории два столбца, которые необходимы нам для дальнейшего анализа. Применили методы value_counts(), apply(), def-if-retutn.

Общий вывод по ПЕРЕРАБОТКЕ ДАННЫХ: 
1. обработали пропуска
2. удалили дубликаты
3. выделили леммы в столбце "purpose"
4. категоризировали данные

<div style="border:solid green 2px; padding: 20px"> <h1 style="color:green; margin-bottom:20px">Комментарий наставника</h1>

Категоризация проведена верно

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?
Гипотеза: зависимость между наличием детей и возвратом кредита в срок есть, люди с детьми отдают кредит с большей вероятностью во время.  

In [38]:
#Для ответа на вопрос нам потребуется рассмотреть два столбца: children_category и debt. 
children_pivot = data.pivot_table(index = ['children_category'], columns = 'debt', values = 'purpose_group', aggfunc='count')
children_pivot['ratio'] = children_pivot[1] / children_pivot[0] * 100
children_pivot
print(children_pivot)

debt                   0     1      ratio
children_category                        
есть дети           6686   678  10.140592
нет детей          13044  1063   8.149341


### Вывод
Мы видим, что люди, у которых нет детей, отдают кредиты хуже, нежели, чем люди с детьми (неважно какое количество детей). 
Возможно, потому что люди с детьми более обязательные с повышенным чувством ответственности. Гипотеза подтвердилась.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?
Гипотеза: семейные люди с большей вероятностью отдадут кредит в срок, нежели люди без семьи. 

In [39]:
#рассмотрим два столбца: family_status и debt
family_status_pivot = data.pivot_table(index = ['family_status'], columns = 'debt', values = 'purpose_group', aggfunc='count')
family_status_pivot['ratio'] = family_status_pivot[1] / family_status_pivot[0] * 100
family_status_pivot
print(family_status_pivot)

debt                       0    1      ratio
family_status                               
в разводе               1110   85   7.657658
вдовец / вдова           896   63   7.031250
гражданский брак        3775  388  10.278146
женат / замужем        11413  931   8.157364
не женат / не замужем   2536  274  10.804416


### Вывод

Мы наблюдаем, что самые низкие показатели по возврату кредита в срок у тех, кто в разводе и вдов/вдовцов. Возможно, потому что отсуствует кормилец в семье, меньше общий доход (так как, когда два человека в семье зарабатывают, вероятность возврата кредита в срок гораздо выше). Гипотеза подтвердилась. 

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
Гипотеза: люди с высоким уровнем дохода с большей вероятностью будут возвращать кредит в срок, нежели люди с низким уровнем дохода. 

In [40]:
income_category_pivot = data.pivot_table(index = ['income_category'], columns = 'debt', values = 'purpose_group', aggfunc='count')
income_category_pivot['ratio'] = income_category_pivot[1] / income_category_pivot[0] * 100
income_category_pivot
print(income_category_pivot)

debt                             0    1     ratio
income_category                                  
высокий уровень дохода        8750  831  9.497143
низкий уровень дохода         2153  184  8.546215
очень высокий уровень дохода  5944  478  8.041723
средний уровень дохода        2883  248  8.602151


### Вывод

Есть несущественная разница влияния уровня дохода на возврат кредита в срок. Стоит отметить, что хуже всего на возврат кредита в срок сказывается очень высокий уровень дохода, на втором месте идет - низкий уровень дохода. Возможно, поэтому уровень дохода очень высокий, потому что не возвращают кредит в срок =). 
Почему люди с низким уровнем дохода возвращают кредит в срок хуже, в целом объяснимо. 
Гипотеза подтвердилась наполовину. 

- Как разные цели кредита влияют на его возврат в срок?
Гипотеза: цели влияют на кредит возврата в срок. 

In [41]:
purpose_group_pivot = data.pivot_table(index = ['purpose_group'], columns = 'debt', values = 'total_income', aggfunc='count')
purpose_group_pivot['ratio'] = purpose_group_pivot[1] / purpose_group_pivot[0] * 100
purpose_group_pivot
print(purpose_group_pivot)

debt                       0    1      ratio
purpose_group                               
автокредит              3905  403  10.320102
ипотека                10032  782   7.795056
кредит на образование   3644  370  10.153677
кредит на свадьбу       2149  186   8.655188


### Вывод

Наименьшие показатели возврата кредита в срок мы видим у следующих целей: ипотека и кредит на свадьбу. 
Кредит на свадьбу - можно охарактеризовать как потребительский, скорее всего его к нему относятся не совсем серьезно, да и на свадьбу брать кредит - не очень серьезная история. 
Ипотека - достаточно рисковая сделка, большие вложения, которыми, скорее всего, не всегда располагают люди, берущие такого рода кредит. 

<div style="border:solid green 2px; padding: 20px"> <h1 style="color:green; margin-bottom:20px">Комментарий наставника</h1>

Все выводы верны, код написан правильно, радует, что используешь сводные таблицы)


### Шаг 4. Общий вывод

Результаты исследования показали, что мы наблюдаем зависимость между семейным положением, наличием детей, уровнем дохода и целями на кредит на возврат кредита в срок. 
- Наиболее повышенны риски по невозврату кредитов наблюдаются у вдовцов/вдов и у разведенных, скорее всего потому, что люди оказались в непростой ситуации, отсутсвие кормильца и т.д. 
- Также следует обратить внимание на цели кредита, а именно кредит на свадьбу и ипотеку. Ипотека - очень крупная сделка с банком с больщим количествои рисков, а кредит на свадьбу - не очень ликвидный кредит. 
- Наличие детей влияет не кардинально на возврат кредита в срок, это не тот пункт на котором стоит заострять внимание. 
- Уровень дохода также влияет на возврат кредита в срок. 

<div style="border:solid green 2px; padding: 20px"> <h1 style="color:green; margin-bottom:20px">Комментарий наставника</h1>

Хороший вывод, как совет на будущее - можешь в выводе писать рекомендации по использованию данного анализа


### Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение, какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено, по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение, какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение, какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.

<div style="border:solid  green  2px; padding: 20px"> <h1 style="color: green ; margin-bottom:20px">Комментарий наставника</h1>

#### Код

Всё отлично. Из того, что очень порадовало - соблюдена структура проекта, шаги из задания обозначены и выполнены последовательно, код написан аккуратно, используются комментарии к коду, быстро можно понять, какие операции выполняют сложные конструкции. В качестве совета предлагаю глубже изучить и начать чаще применять конструкцию try-except в решении задачи — это улучшит отказоустойчивость кода и обезопасит код от поломок в будущем, а так же изучить средства для построения изображений в Python для более глубокого понимания данных.
#### Выводы

У тебя отлично получается анализировать сложные данные, выдвигать корректные гипотезы и проверять свои выводы на возможность соответствия реальности. Видно глубокое понимание сути проведённого анализа. Было очень интересно проверять твой проект и следить за твоей мыслью, так держать!)


<div style="border:solid  green  2px; padding: 20px"> <h1 style="color: green ; margin-bottom:20px">Комментарий наставника</h1>
Работа принята!) Очень рад, что у тебя всё получилось и надеюсь тебе понравится весь курс, будь усердна и у тебя всё получится! Удачи в будущих проектах)